In [6]:
# 필요한 패키지 설치
!pip install requests selenium beautifulsoup4 webdriver_manager


In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time


In [14]:
# Selenium 설정
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # 브라우저를 표시하지 않고 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# FAQ 페이지 URL
url = "https://www.samsungsvc.co.kr/solution?category=10471&product=10502"

# 웹페이지 로딩
driver.get(url)
time.sleep(3)  # 페이지 로딩 대기


In [15]:
# FAQ 데이터 추출 함수
def extract_faq_data(driver):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    faq_items = soup.find_all('div', class_='accordion-item')  # FAQ 아이템을 포함하는 div 클래스
    
    faq_list = []
    for item in faq_items:
        # 질문과 답변 태그 찾기
        question_tag = item.find('h3')
        answer_tag = item.find('p')
        
        # 질문과 답변 텍스트 추출 (없으면 기본값 사용)
        question = question_tag.get_text(strip=True) if question_tag else '[질문 없음]'
        answer = answer_tag.get_text(strip=True) if answer_tag else '[답변 없음]'
        
        # 결과 저장
        faq_list.append({
            'question': question,
            'answer': answer
        })
        
        # 콘솔에 출력
        print(f"\nQ: {question}")
        print(f"A: {answer}")
        print("-" * 80)
    
    return faq_list


In [16]:
# FAQ 데이터 추출 실행
faq_data = extract_faq_data(driver)

# 브라우저 종료
driver.quit()

In [17]:
# 1. 단순 출력 방식
def print_faq_results(faq_data):
    print("\n=== FAQ 크롤링 결과 ===")
    print(f"총 {len(faq_data)}개의 FAQ 항목이 수집되었습니다.\n")

    for i, faq in enumerate(faq_data, 1):
        print(f"[{i}번째 FAQ]")
        print(f"Q: {faq['question']}")
        print(f"A: {faq['answer']}")
        print("-" * 80)


# 2. pandas DataFrame으로 변환하여 보기
import pandas as pd


def save_to_dataframe(faq_data):
    df = pd.DataFrame(faq_data)
    print("\n=== DataFrame 형식으로 보기 ===")
    print(df)
    return df


# 3. Excel 파일로 저장
def save_to_excel(df, filename='samsung_faq_results.xlsx'):
    df.to_excel(filename, index=False)
    print(f"\nExcel 파일이 저장되었습니다: {filename}")


# 실행
# 1. 콘솔에서 바로 확인
print_faq_results(faq_data)

# 2. DataFrame으로 변환
df = save_to_dataframe(faq_data)

# 3. Excel 파일로 저장
save_to_excel(df)


=== FAQ 크롤링 결과 ===
총 0개의 FAQ 항목이 수집되었습니다.


=== DataFrame 형식으로 보기 ===
Empty DataFrame
Columns: []
Index: []

Excel 파일이 저장되었습니다: samsung_faq_results.xlsx


<ipython-input-17-8c8228f13cc3>:26: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  df.to_excel(filename, index=False)
